# T2M (2-meter Temperature) GRIB2 File - Exploratory Data Analysis

This notebook is designed to explore and understand the structure of GRIB2 files containing 2-meter temperature data from ICON-D2 model for Germany.

**Goal**: Unfold and understand every aspect of the GRIB2 file in a structured way, similar to how we would explore a pandas DataFrame.

**File to analyze**: `../data/icon-d2_germany_regular-lat-lon_single-level_2025110700_000_2d_t_2m.grib2`

We'll proceed step-by-step to:
1. Load the file
2. Inspect how many parameters are present
3. Understand each parameter in detail
4. Explore relationships between parameters
5. Visualize the data structure and content

## 1. Import Required Libraries

Let's start by importing the necessary libraries:
- **xarray**: For working with multi-dimensional labeled arrays (think of it as pandas for N-dimensional data)
- **cfgrib**: Backend engine for xarray to read GRIB files
- **pandas**: For familiar tabular data operations when we convert data
- **numpy**: For numerical operations
- **matplotlib**: For visualization

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For better display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

## 2. Load the GRIB2 File

We'll load the GRIB2 file using xarray with the cfgrib engine. This will give us a Dataset object, which is similar to having multiple related DataFrames with shared indices.

In [ ]:
import bz2

# Load the GRIB2 file
file_path = '../data/t2m/icon-d2_germany_regular-lat-lon_single-level_2025110700_000_2d_t_2m.grib2.bz2'
decompressed_path = '../data/t2m/icon-d2_germany_regular-lat-lon_single-level_2025110700_000_2d_t_2m.grib2'  # Temporary decompressed file

# Decompress the file
with bz2.open(file_path, 'rb') as f_in, open(decompressed_path, 'wb') as f_out:
    f_out.write(f_in.read())

# Open the decompressed file using xarray
ds = xr.open_dataset(decompressed_path, engine='cfgrib')

print(f"File loaded successfully!")
print(f"Dataset type: {type(ds)}")


## 3. First Look at the Dataset

Let's take our first look at what's inside the dataset. Think of this as similar to running `df.info()` or `df.head()` in pandas.

In [ ]:
ds

## 5. Convert Dataset to Pandas DataFrames

Now let's convert the xarray Dataset into pandas DataFrames. This will make it easier for you to analyze each parameter using familiar pandas operations.

In xarray, each parameter (data variable) can be converted to a DataFrame, where:
- **Index**: Will be the coordinate dimensions (latitude, longitude, time, etc.)
- **Columns**: Will include the parameter values and coordinate values
- **Shape**: Will be flattened from the multi-dimensional array

In [ ]:
# Convert the entire dataset to a pandas DataFrame
# This will create a multi-index DataFrame with all parameters
df_full = ds.to_dataframe()

print("Full dataset converted to DataFrame:")
print(f"Shape: {df_full.shape}")
print(f"Columns: {list(df_full.columns)}")
print(f"Index levels: {df_full.index.names}")
print("\nFirst few rows:")
df_full.info()

In [ ]:
df_full.loc[df_full["t2m"].notnull()]

## 6. Analyze Each Parameter Individually

Let's create separate DataFrames for each parameter and examine them one by one. This will help us understand what each parameter represents.

In [ ]:
# Create individual DataFrames for each parameter
parameter_dfs = {}

for var_name in ds.data_vars:
    # Convert each data variable to DataFrame
    param_df = ds[var_name].to_dataframe()
    parameter_dfs[var_name] = param_df

    print(f"\n=== Parameter: {var_name} ===")
    print(f"Shape: {param_df.shape}")
    print(f"Data type: {param_df[var_name].dtype}")
    print(f"Has NaN values: {param_df[var_name].isna().any()}")
    print(f"Value range: {param_df[var_name].min():.4f} to {param_df[var_name].max():.4f}")

    # Show first few rows
    print("First 5 rows:")
    print(param_df.head())
    print("-" * 50)

## 7. Examine Parameter Metadata

Let's look at the detailed metadata for each parameter to understand what they represent, their units, and other important attributes.

In [ ]:
# Examine metadata for each parameter
for var_name in ds.data_vars:
    var = ds[var_name]
    print(f"\n=== Metadata for: {var_name} ===")

    # Basic attributes
    print(f"Dimensions: {var.dims}")
    print(f"Shape: {var.shape}")
    print(f"Data type: {var.dtype}")

    # GRIB-specific attributes
    attrs = var.attrs
    if attrs:
        print("Attributes:")
        for key, value in attrs.items():
            print(f"  {key}: {value}")
    else:
        print("No attributes found")

    print("-" * 50)

## 8. Explore Coordinate Systems

Let's examine the coordinate variables (latitude, longitude, time, etc.) that define the spatial and temporal structure of our data.

In [ ]:
# Examine coordinate variables
print("Coordinate Variables:")
print("=" * 50)

for coord_name in ds.coords:
    coord = ds.coords[coord_name]
    print(f"\n=== Coordinate: {coord_name} ===")
    print(f"Dimensions: {coord.dims}")
    print(f"Shape: {coord.shape}")
    print(f"Data type: {coord.dtype}")
    print(f"Range: {coord.min().values} to {coord.max().values}")

    # Show first few values
    print(f"First 5 values: {coord.values}")

    # Attributes
    if coord.attrs:
        print("Attributes:")
        for key, value in coord.attrs.items():
            print(f"  {key}: {value}")

print("\n" + "=" * 50)
print("Dataset dimensions summary:")
for dim_name, size in ds.dims.items():
    print(f"  {dim_name}: {size}")

## 9. Basic Statistical Analysis

Let's perform some basic statistical analysis on each parameter to understand their distributions and characteristics.

In [ ]:
# Statistical summary for each parameter
for var_name in ds.data_vars:
    param_data = ds[var_name]
    print(f"\n=== Statistics for: {var_name} ===")

    # Basic statistics
    print(f"Mean: {param_data.mean().values:.4f}")
    print(f"Std: {param_data.std().values:.4f}")
    print(f"Min: {param_data.min().values:.4f}")
    print(f"Max: {param_data.max().values:.4f}")
    print(f"Median: {param_data.median().values:.4f}")

    # Quantiles
    print("Quantiles:")
    for q in [0.25, 0.5, 0.75]:
        print(f"  {q*100:.0f}%: {param_data.quantile(q).values:.4f}")

    print("-" * 30)

## 12. Spatial Visualization - Plotting Coordinates on Map

Let's start simple: plot all the latitude and longitude coordinate pairs on a map to visualize the spatial coverage of your GRIB2 data.

In [ ]:
# Set up the plotting environment
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Get coordinate information
lats = ds.latitude.values
lons = ds.longitude.values

print(f"Latitude range: {lats.min():.2f}° to {lats.max():.2f}°")
print(f"Longitude range: {lons.min():.2f}° to {lons.max():.2f}°")
print(f"Grid resolution: {len(lats)} x {len(lons)} points")
print(f"Total grid points: {len(lats) * len(lons)}")

# Create meshgrid of all coordinate points
lon_mesh, lat_mesh = np.meshgrid(lons, lats)

print(f"Meshgrid shape: {lon_mesh.shape}")
print("Ready to plot coordinates!")

### Simple Coordinate Plot

Let's plot all the latitude and longitude coordinate pairs as points on a map.

In [ ]:
# Simple scatter plot of all coordinate points
plt.figure(figsize=(12, 8))

plt.scatter(lon_mesh, lat_mesh, c='blue', s=2, alpha=0.6, label='Grid Points')

# Set labels and title
plt.xlabel('Longitude (°)', fontsize=12)
plt.ylabel('Latitude (°)', fontsize=12)
plt.title('GRIB2 Grid Coordinates - Germany\nAll Latitude/Longitude Points', fontsize=14, fontweight='bold')

# Add grid
plt.grid(True, alpha=0.3)

# Add some reference information
stats_text = f'''Grid Statistics:
• Total points: {len(lats) * len(lons):,}
• Latitude points: {len(lats)}
• Longitude points: {len(lons)}
• Lat range: {lats.min():.2f}° to {lats.max():.2f}°
• Lon range: {lons.min():.2f}° to {lons.max():.2f}°'''

plt.text(0.02, 0.98, stats_text, transform=plt.gca().transAxes,
         verticalalignment='top', fontsize=10, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

print("✅ Successfully plotted all coordinate points!")
print(f"Each blue dot represents one latitude/longitude measurement location.")
print(f"This shows the complete spatial coverage of your GRIB2 data.")

### Understanding the Grid

Let's examine what this coordinate plot tells us about your data:

1. **Spatial Coverage**: The blue dots show where measurements exist
2. **Grid Structure**: Notice the regular pattern - this is a structured grid
3. **Resolution**: The density of points shows the spatial resolution
4. **Geographical Area**: The extent covers Germany and surrounding areas

**Key Observations:**
- The points form a regular rectangular grid
- Each intersection of latitude and longitude lines has a data point
- The grid covers the entire domain without gaps
- This is typical for numerical weather prediction models

In [ ]:
# Examine the coordinate values more closely
print("\nFirst 10 latitude values:")
print(lats[:10])

print("\nFirst 10 longitude values:")
print(lons[:10])

print(f"\nLatitude spacing: {np.diff(lats[:5]).mean():.4f}°")
print(f"Longitude spacing: {np.diff(lons[:5]).mean():.4f}°")

print("\nThis shows:")
print("- Latitude decreases from north to south (typical for meteorological data)")
print("- Longitude increases from west to east")
print("- Regular spacing indicates a structured grid")

## 13. Plot Coordinates on Geographical Map

Now let's plot the coordinates on an actual geographical map with proper map features like coastlines, country borders, and geographical context.

In [ ]:
# Try to use Cartopy for proper geographical maps
try:
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    print("✅ Cartopy is available - creating geographical map!")

    # Create figure with PlateCarree projection (appropriate for lat/lon data)
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

    # Add geographical features
    ax.add_feature(cfeature.COASTLINE, linewidth=1, edgecolor='black')
    ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='gray')
    ax.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.3)
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue', alpha=0.3)

    # Plot the coordinate points
    ax.scatter(lon_mesh, lat_mesh, c='red', s=1, alpha=0.7,
               transform=ccrs.PlateCarree(), label='GRIB2 Grid Points')

    # Set map extent to focus on Germany and surroundings
    ax.set_extent([lons.min()-1, lons.max()+1, lats.min()-1, lats.max()+1],
                  crs=ccrs.PlateCarree())

    # Add gridlines
    ax.gridlines(draw_labels=True, alpha=0.3)

    # Add title and labels
    ax.set_title('GRIB2 Grid Points on Geographical Map\nGermany - ICON-D2 Model',
                 fontsize=16, fontweight='bold', pad=20)

    # Add statistics text
    stats_text = f'''Grid Statistics:
• Total points: {len(lats) * len(lons):,}
• Lat: {lats.min():.2f}° to {lats.max():.2f}°
• Lon: {lons.min():.2f}° to {lons.max():.2f}°
• Resolution: {len(lats)} × {len(lons)}'''

    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes,
            verticalalignment='top', fontsize=10,
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    plt.legend(loc='lower right')
    plt.tight_layout()
    plt.show()

    print("✅ Successfully created geographical map with Cartopy!")

except ImportError:
    print("⚠️  Cartopy not available. Let's install it first...")
    print("Run this command in your terminal:")
    print("pip install cartopy")
    print()
    print("Or if using conda:")
    print("conda install -c conda-forge cartopy")
    print()
    print("After installation, re-run this cell for a proper geographical map.")
    print()
    print("For now, let's create a simple map-like visualization...")

    # Fallback: Create a map-like visualization without Cartopy
    plt.figure(figsize=(12, 10))

    # Create a simple map-like background
    plt.fill_between([lons.min(), lons.max()], [lats.min(), lats.min()], [lats.max(), lats.max()],
                    color='lightblue', alpha=0.3, label='Ocean')
    plt.fill_between([lons.min(), lons.max()], [lats.min(), lats.max()], [lats.max(), lats.max()],
                    color='lightgray', alpha=0.3, where=(lons >= lons.min()) & (lons <= lons.max()),
                    label='Land')

    # Plot grid points
    plt.scatter(lon_mesh, lat_mesh, c='red', s=2, alpha=0.8, label='GRIB2 Grid Points')

    # Add some major cities for reference
    cities = {
        'Berlin': (13.4, 52.5),
        'Munich': (11.6, 48.1),
        'Hamburg': (10.0, 53.5),
        'Cologne': (6.96, 50.94),
        'Frankfurt': (8.68, 50.11)
    }

    for city, (lon, lat) in cities.items():
        plt.scatter(lon, lat, c='blue', s=50, marker='*', edgecolors='black', linewidth=1)
        plt.annotate(city, (lon, lat), xytext=(5, 5), textcoords='offset points',
                    fontsize=9, bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))

    plt.xlabel('Longitude (°)', fontsize=12)
    plt.ylabel('Latitude (°)', fontsize=12)
    plt.title('GRIB2 Grid Points - Germany\n(Simple Map View)', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.legend(loc='upper right')

    # Add statistics
    stats_text = f'''Grid Statistics:
• Total points: {len(lats) * len(lons):,}
• Lat: {lats.min():.2f}° to {lats.max():.2f}°
• Lon: {lons.min():.2f}° to {lons.max():.2f}°'''

    plt.text(0.02, 0.98, stats_text, transform=plt.gca().transAxes,
             verticalalignment='top', fontsize=10, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    plt.tight_layout()
    plt.show()

    print("✅ Created simple map-like visualization!")
    print("💡 For better geographical maps, install Cartopy:")
    print("   pip install cartopy")

### Alternative: Interactive Map with Plotly

If you prefer an interactive map, here's how to create one with Plotly:

In [ ]:
# Try to create an interactive map with Plotly
try:
    import plotly.express as px
    import plotly.graph_objects as go

    print("✅ Plotly available - creating interactive map!")

    # Flatten the meshgrid for plotting
    lons_flat = lon_mesh.flatten()
    lats_flat = lat_mesh.flatten()

    # For performance, subsample the data (plot every 10th point)
    subsample_factor = 10
    lons_sample = lons_flat[::subsample_factor]
    lats_sample = lats_flat[::subsample_factor]

    print(f"Plotting {len(lons_sample):,} points (subsampled from {len(lons_flat):,} total)")

    # Create interactive scatter plot on map
    fig = px.scatter_mapbox(
        lat=lats_sample,
        lon=lons_sample,
        zoom=6,
        center=dict(lat=lats.mean(), lon=lons.mean()),
        title="GRIB2 Grid Points - Germany (Interactive Map)",
        mapbox_style="open-street-map"
    )

    # Update marker appearance
    fig.update_traces(
        marker=dict(size=4, color='red', opacity=0.8),
        mode='markers'
    )

    # Add some statistics as annotations
    stats_text = f"Grid Points Shown: {len(lons_sample):,}<br>" + \
                f"Total Grid Points: {len(lats) * len(lons):,}<br>" + \
                f"Lat Range: {lats.min():.2f}° to {lats.max():.2f}°<br>" + \
                f"Lon Range: {lons.min():.2f}° to {lons.max():.2f}°<br>" + \
                f"Subsampling: Every {subsample_factor}th point"

    fig.add_annotation(
        text=stats_text,
        xref="paper", yref="paper",
        x=0.02, y=0.98,
        showarrow=False,
        bgcolor="white",
        bordercolor="black",
        borderwidth=1
    )

    # Try to show the figure
    try:
        fig.show()
        print("✅ Interactive map created successfully!")
        print("💡 You can zoom, pan, and explore the grid points interactively!")
    except Exception as e:
        print(f"⚠️  Could not display interactive map: {e}")
        print("This might be due to:")
        print("- Running in a non-interactive environment")
        print("- Network connectivity issues")
        print("- Large dataset size")
        print()
        print("Try running this in a Jupyter notebook or JupyterLab for best results.")

except ImportError:
    print("⚠️  Plotly not available for interactive maps.")
    print("To install: pip install plotly")
    print("Or with uv: uv add plotly")
    print("Then re-run this cell for an interactive map experience.")
except Exception as e:
    print(f"⚠️  Error creating Plotly map: {e}")
    print("Common solutions:")
    print("1. Check your internet connection for map tiles")
    print("2. Try a different mapbox_style (e.g., 'carto-positron')")
    print("3. Reduce the subsample_factor for fewer points")
    print("4. Use the matplotlib fallback instead")